In [29]:
import dwavebinarycsp as dbc
import minorminer
from dwave.system.samplers import DWaveSampler 
import dimod
import math
import re
import statistics as stat

my_solver = 'DW_2000Q_2'
my_token = 'PREM-426a488702253a4cd34901800bf165bb55582e3c'

def toDecimal(sample, prefix):
    p = re.compile('%s([0-9]+)' % prefix)
    return sum([2**int(p.match(k)[1])*v for (k,v) in sample.items() if p.match(k)])+1


In [30]:
#P = 5*7 #35, 6 bits
P = 11*13 #143, 8 bits
#P = 29*31 #899, 10 bits
#P = 113*127 #14351, 14 bits
#P = 241*251 #60491, 16 bits
#P = 503*509 #256027 18 bits

N = (math.floor(math.log2(P))+1)//2
csp = dbc.factories.multiplication_circuit(N)
bqm = dbc.stitch(csp, min_classical_gap=.1)

#fix output bits
for i in range(2*N): bqm.fix_variable('p%d'%i, (P>>i)&1)
#fix low bit of factors since they are always odd
bqm.fix_variable('a0', 1)
bqm.fix_variable('b0', 1)
#bqm.fix_variable('a3', 1)
#bqm.fix_variable('b3', 1)

In [26]:
sampler = DWaveSampler(solver=my_solver, token=my_token)
_, target_edgelist, target_adjacency = sampler.structure

embedding = minorminer.find_embedding(bqm.quadratic, target_edgelist)
if bqm and not embedding:
    raise ValueError("no embedding found")

In [27]:
bqm_embedded = dimod.embed_bqm(bqm, embedding, target_adjacency, 4)

print(len(list(embedding.keys())), "logical qubits")
phys = list()
for _, v in embedding.items():
    phys += v
print(len(phys), "physical qubits")

chain_lens = [len(v) for _, v in embedding.items()]
print('Longest chain: ', max(chain_lens))
print('Shortest chain: ', min(chain_lens))
print('Average chain: ', stat.mean(chain_lens))

36 logical qubits
120 physical qubits
Longest chain:  7
Shortest chain:  1
Average chain:  3.3333333333333335


In [28]:
response = sampler.sample(bqm_embedded, num_reads = 1000, answer_mode = 'histogram')
response = dimod.unembed_response(response, embedding, source_bqm=bqm)

row_format ="{:>8}" * 5
print(row_format.format('p', 'q', 'Correct', 'Energy', 'Count'))
for sample in response.data(fields=['sample', 'energy', 'num_occurrences']):
    p = toDecimal(sample[0], 'a')
    q = toDecimal(sample[0], 'b')
    print(row_format.format(p, q, p*q==P, sample[1], sample[2]))
print(sample[0])

       p       q Correct  Energy   Count
      11      13       1     0.0       4
       9      15       0     1.0       1
      11      13       1     2.0       1
      11      13       1     2.0       1
      11      13       1     2.0       2
       9      15       0     2.0       1
      11      13       1     2.0       1
      11      13       1     2.0       1
      13      11       1     2.0       2
       9      15       0     2.0       1
      15       9       0     2.0       1
      15       9       0     2.0       1
      13      11       1     2.0       1
      15      13       0     3.0       1
      15      13       0     3.0       1
      11      13       1     3.0       1
      15      13       0     3.0       1
      13      15       0     3.0       1
      13      15       0     3.0       3
       9      15       0     3.0       1
       9      15       0     3.0       1
      15      15       0     3.0       1
       9      15       0     3.0       3
      15      13